This notebook is responsible for visualizing the data from the Github API, which should be provided in a serialized format.

It must be possible to run this notebook using jupyterlite.

If you do not see the newest version of the notebook, you have to "delete" it in your jupyterlite instance, as your local version is overriding the remote version provided by the deployment.

(jupyterlite) We need to manually install dependencies for the front-end, even though they are also defined in the `requirements.txt`.

### Troubleshooting:

#### Even after reloading (deep-refreshing) the page, a notebook is not updated

This likely occurred because the backend and frontend of jupyterlite are out of sync.
I am not yet sure why this happens even after a deep-refresh of the page, as this should update the frontend according to the jupyterlite documentation.
As a workaround, delete the cache/cookies for the page and reload it.
Note that you this will reset all notebooks to their versions saved on Github, so download your notebooks if necessary.
You will also have to re-run the notebooks.

An additional necessary step may be to "delete" the files in question in jupyterlite, which should refresh them with the version on Github.

#### The `commits.json` cannot be found when loading

Same reasoning & workaround as above.

In [ ]:
%pip install plotly ipywidgets

In [85]:
import json
import datetime
import numpy as np
import plotly
import plotly.express as px
import ipywidgets
from IPython import display

In [3]:
# Read the commit data from the 'commits.json'
with open('commits.json') as f:
	commits_including_merges = json.load(f)

In [4]:
# Convert all the commit dates to datetime objects within the commit for easier handling later on
for commit in commits_including_merges:
	commit['commit']['author']['date'] = datetime.datetime.strptime(commit['commit']['author']['date'], '%Y-%m-%dT%H:%M:%SZ')

In [5]:
# Exclude merge commits, i.e. commits with more than one parent
commits = [commit for commit in commits_including_merges if len(commit['parents']) == 1]
print(f'There are a total of {len(commits_including_merges)} commits in the repository, of which {len(commits)} are not merge commits.')

There are a total of 1007 commits in the repository, of which 712 are not merge commits.


In [6]:
# Exclude commits without an "author" object
# e.g. sha 9a650be18f07d01a55d1c77d55f74dde7b24eb55 (in our test case, there are number of commits with "author: null" or "author: {}"")
commits = [commit for commit in commits if (commit['author'] and len(commit['author']) > 0)]

This cell handles the number and date ranges of your sprints.
Use the slider to change the number of sprints, and then use the text field and date picker to set a custom name and date range for each sprint.
The number of displayed date pickers will change automatically when changing the number of sprints.
Names and dates for "removed" sprints will be kept, but not displayed.

In [7]:
# Setting up the number & date ranges of sprints
num_sprints_output = ipywidgets.Output()

num_sprints = ipywidgets.IntSlider(min=1, max=10, step=1, description='No. of sprints')

num_sprints_output.append_display_data(num_sprints)
num_sprints_output.append_stdout('Set a name and the start and end dates for each sprint below:')

sprint_names = []
start_dates = []
end_dates = []

# Initialize the widgets for the first sprint
sprint_names.append(ipywidgets.Text(value=f'Sprint 1', placeholder='Set the name for this Sprint', description='Sprint Name:'))
start_dates.append(ipywidgets.DatePicker(description='Start Date'))
end_dates.append(ipywidgets.DatePicker(description='End Date'))
num_sprints_output.append_display_data(sprint_names[0])
num_sprints_output.append_display_data(start_dates[0])
num_sprints_output.append_display_data(end_dates[0])

# Automatically add/remove the widgets based on the number of sprints
def update_sprints(change):
	if change['type'] == 'change' and change['name'] == 'value':
		if change['new'] > change['old']:
			# Only add a new widget if we do not have one with this "number" already
			if len(sprint_names) < change['new']:
				sprint_names.append(ipywidgets.Text(value=f'Sprint {change["new"]}', placeholder='Set the name for this Sprint', description='Sprint Name:'))
				start_dates.append(ipywidgets.DatePicker(description='Start Date'))
				end_dates.append(ipywidgets.DatePicker(description='End Date'))
			# Display the necessary widgets for the new sprint
			num_sprints_output.append_display_data(sprint_names[change["new"] - 1])
			num_sprints_output.append_display_data(start_dates[change["new"] - 1])
			num_sprints_output.append_display_data(end_dates[change["new"] - 1])
		elif change['new'] < change['old']:
			# Remove the widgets for the removed sprint (workaround, as clear_output() will not work here, as we are using append_display_data() instead of the 'with out:' syntax)
			num_sprints_output.outputs = num_sprints_output.outputs[:-3]

num_sprints.observe(update_sprints, names='value')

display.display(num_sprints_output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "IntSlider(value=1, description='No. of…

This cell handles setting up teams and team members.

Disclaimer: The Github API does not always return the username of the author of a commit, but rather the name of the author.
This has to do with the various ways one can commit to Github, e.g. by using the Github website itself, the username will not be displayed. 
So, in its current form, the heatmap generated will not always include all commits by all team members.

In [8]:
teams_output = ipywidgets.Output()

num_teams = ipywidgets.IntSlider(min=1, max=10, step=1, description='No. of teams')

teams_output.append_display_data(num_teams)
teams_output.append_stdout('For each team, set a name and upload a JSON file with an Array of team member names below (more formats to follow):')

team_names = []
team_files = []
team_members = []

# Handler for the file upload widgets
def handle_upload(change):
	# Only handle the upload if the file is not empty
	if change['new']:
		# Get the team index from the name of the widget
		team_index = int(change['owner'].description.split(' ')[2]) - 1
		# Get the file name and content
		uploaded_file = next(iter(team_files[team_index].value))
		with open(uploaded_file.name, 'wb') as f:
			f.write(uploaded_file.content)
		with open(uploaded_file.name) as f:
			team_members[team_index] = json.load(f)
		print(f'Uploaded file {uploaded_file.name} for team {team_names[i].value}')
		# Print the team members
		print(f'Team members:')
		print("\n".join(team_members[team_index]))
		print()

# Initialize the widgets for the first team
team_names.append(ipywidgets.Text(value=f'Team 1', placeholder='Set the name for this Team', description='Team Name:'))
team_files.append(ipywidgets.FileUpload(description=f'Upload Team 1', accept='.json'))
team_members.append([])
teams_output.append_display_data(team_names[0])
teams_output.append_display_data(team_files[0])

# Register the upload handler for the first team
team_files[0].observe(handle_upload, names='value')

# Automatically add/remove the widgets based on the number of teams
def update_teams(change):
	if change['type'] == 'change' and change['name'] == 'value':
		if change['new'] > change['old']:
			# Only add a new widget if we do not have one with this "number" already
			if len(team_names) < change['new']:
				team_names.append(ipywidgets.Text(value=f'Team {change["new"]}', placeholder='Set the name for this Team', description='Team Name:'))
				team_files.append(ipywidgets.FileUpload(description=f'Upload Team {change["new"]}', accept='.json'))
				team_members.append([])
				# Register the upload handler for the new team
				team_files[-1].observe(handle_upload, names='value')
			# Display the necessary widgets for the new team
			teams_output.append_display_data(team_names[change["new"] - 1])
			teams_output.append_display_data(team_files[change["new"] - 1])
		elif change['new'] < change['old']:
			# Remove the widgets for the removed team (workaround, as clear_output() will not work here, as we are using append_display_data() instead of the 'with out:' syntax)
			teams_output.outputs = teams_output.outputs[:-2]

num_teams.observe(update_teams, names='value')

display.display(teams_output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "IntSlider(value=1, description='No. of…

In [9]:
# Only for local use in VSCode, as the upload widget does not work there:
class MockFileUpload:
    def __init__(self, value):
        self.value = value

team_names = [
    MockFileUpload("Team AP"),
    MockFileUpload("Team FN")
]

team_members = [
    [
		"A-Persitzky",
		"antonykamp",
		"CR1337",
		"LucasDerReisende",
		"SaturnHafen",
		"Lietze"
    ], 
    [
		"gwauge",
		"Arkinul",
		"MatthiasCr",
		"MaxSpeer",
		"Punguitius",
		"Glitterrosie",
		"simon-weissmueller"
    ]
]

This cell contains the logic for the commit heatmap.
For a given time interval, the number of commits per day and hour of the day is displayed in a heatmap.

In [110]:
def get_time_of_day(date):
	# Get the hour of the day
	hour = date.hour
	# Return the time of day
	if hour <= 7 or hour >= 22:
		return 0 # "night"
	elif hour <= 12:
		return 1 # "morning"
	elif hour <= 17:
		return 2 # "afternoon"
	else:
		return 3 # "evening"

# Plot a heatmap for the commit timings
def plot_commit_heatmap(commits, team_members=None, start_date=None, end_date=None, time_mode="time_of_day"):
	# Convert start and and dates to datetime objects
	if start_date is not None:
		start_date = datetime.datetime.combine(start_date, datetime.datetime.min.time())
	if end_date is not None:
		end_date = datetime.datetime.combine(end_date, datetime.datetime.max.time())

	# Only include commits authored by one of the team members
	if team_members is not None:
		commit_dates = [commit['commit']['author']['date'] for commit in commits if (commit['author']['login'] in team_members)]
	else:
		commit_dates = [commit['commit']['author']['date'] for commit in commits]
	
	# Filter the dates to only include those between the start and end dates
	commit_dates = [date for date in commit_dates if (start_date is None or date >= start_date) and (end_date is None or date <= end_date)]

	# Get the day of each commit
	days = [date.weekday() for date in commit_dates]

	# Depending on the time_mode, get the hour or time of day for each commit
	if time_mode == 'hour':
		x_vals = [date.hour for date in commit_dates]
		x_axis_title = 'Hour of the day'
		x_axis_tick_vals = np.arange(0, 24, 2)
		x_axis_tick_text = np.arange(0, 24, 2)
		x_bins = 24
	elif time_mode == 'time_of_day':
		x_vals = [get_time_of_day(date) for date in commit_dates]
		x_axis_title = 'Time of day'
		x_axis_tick_vals = np.arange(0, 4)
		x_axis_tick_text = ['Night (22-7)', 'Morning (7-12)', 'Afternoon (12-17)', 'Evening (17-22)']
		x_bins = 4
	
	# Create a plotly heatmap
	fig = px.density_heatmap(x=x_vals, y=days, nbinsx=x_bins, nbinsy=7, histfunc='count', color_continuous_scale='oranges')

	# Set y axis labels
	fig.update_yaxes(tickvals=np.arange(0, 7), ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

	# Set the title and axis labels
	fig.update_layout(title='Commit timings', xaxis_title=x_axis_title, yaxis_title='Day of the week', xaxis_tickvals=x_axis_tick_vals, xaxis_ticktext=x_axis_tick_text)

	return fig

This cell handles the display of the heatmap for the given sprints.
Only one heatmap is displayed at a time, the sprint, team and "time-mode" for which this is done can be selected using the dropdown menus.
If you change the date ranges of the sprints, you *do not* have to re-run this cell.
However, you do need to re-run the cell if you have changed the number of sprints, as this is not currently handled automatically.

In [111]:
# Widget to select the displayed sprint
sprint_selector = ipywidgets.Dropdown(
	options=[(sprint_names[i].value, i) for i in range(num_sprints.value)],
	description='Sprint:'
)
display.display(sprint_selector)

# Widget to select the displayed team
team_selector = ipywidgets.Dropdown(
	options=[('All Authors', -1)] + [(team_names[i].value, i) for i in range(num_teams.value)],
	description='Team:'
)
display.display(team_selector)
selected_team_name = team_names[team_selector.value].value if team_selector.value != -1 else 'All Authors'
selected_team_members = team_members[team_selector.value] if team_selector.value != -1 else None

# Widget to select which time-mode to display
time_mode_selector = ipywidgets.Dropdown(
	options=[('Hour of the day', 'hour'), ('Time of day', 'time_of_day')],
	description='Time mode:'
)
display.display(time_mode_selector)

fig = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value)
fig.update_layout(title=f'Commit timings during sprint \'{sprint_names[sprint_selector.value].value}\' ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value}) for team \'{selected_team_name}\'')
display.display(fig)

def update_sprint(change):
	selected_team_name = team_names[team_selector.value].value if team_selector.value != -1 else 'All Authors'
	selected_team_members = team_members[team_selector.value] if team_selector.value != -1 else None
	fig.data[0].z = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value).data[0].z
	fig.update_layout(title=f'Commit timings during sprint \'{sprint_names[change["new"]].value}\' ({start_dates[change["new"]].value} - {end_dates[change["new"]].value}) for team \'{selected_team_name}\'')

sprint_selector.observe(update_sprint, names='value')

def update_team(change):
	selected_team_name = team_names[team_selector.value].value if team_selector.value != -1 else 'All Authors'
	selected_team_members = team_members[team_selector.value] if team_selector.value != -1 else None
	fig.data[0].z = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value).data[0].z
	fig.update_layout(title=f'Commit timings during sprint \'{sprint_names[sprint_selector.value].value}\' ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value}) for team \'{selected_team_name}\'')

team_selector.observe(update_team, names='value')

def update_time_mode(change):
	selected_team_name = team_names[team_selector.value].value if team_selector.value != -1 else 'All Authors'
	selected_team_members = team_members[team_selector.value] if team_selector.value != -1 else None
	# fig.data[0].y = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value).data[0].y
	fig.data[0].z = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value).data[0].z
	fig.layout = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value).layout
	fig.update_layout(title=f'Commit timings during sprint \'{sprint_names[sprint_selector.value].value}\' ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value}) for team \'{selected_team_name}\'')

time_mode_selector.observe(update_time_mode, names='value')

Dropdown(description='Sprint:', options=(('Sprint 1', 0), ('Sprint 2', 1)), value=0)

Dropdown(description='Team:', options=(('All Authors', -1), ('Team AP', 0)), value=-1)

Dropdown(description='Time mode:', options=(('Hour of the day', 'hour'), ('Time of day', 'time_of_day')), valu…

AttributeError: 'Figure' object has no attribute 'close'